In [5]:
import pystac_client
import planetary_computer
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import timedelta,date,datetime
import matplotlib.pyplot as plt
import dask.dataframe as dd
#from netCDF4 import Dataset

# Order of operations:

1. **dataAnalysis/clean_eclipse_append_cmaq.ipynb : pull in eclipse, make hourly average, match CMAQ to hourly eclipse data**

2. dataAnalysis/clean_eclipse_find_nearest.ipynb : pull in output from #1, drop bad sensors

3. testML/format_input_data.ipynb : pull in output from #2, add in reanalysis data, ndvi, crop to chicago domain

4. dataAnalysis/format_input_shapefiles.ipynb: final form for monthly average data, add in zoning, put onto shapefile

# pull data using Internal API (to get the 5 min values)


In [ ]:
import requests 
import json
import pandas as pd

def get_eclipse_data(devices: list, start_datetime_utc: str, end_datetime_utc: str) -> pd.DataFrame:
    # get data from url
    #devices = str(devices)[1:-1]
    eclipse_raw = []
    for device in devices:
        url = requests.get("https://eclipseprowebapi.azurewebsites.net/EclipseData/GetReadingsForCustomer?CustomerName=Chicago"
                            "&API_KEY=494FE543-82B7-45A2-8C1D-D9209AE45C94"
                            "&BeginDateTime={}&EndDateTime={}"
                            "&DeviceSubset={}".format(start_datetime_utc, end_datetime_utc, device))
        text = url.text
        data_dict_list = json.loads(text)
        if isinstance(data_dict_list, list):
            eclipse_raw += data_dict_list
    eclipse_raw = pd.DataFrame.from_records(eclipse_raw)
    return eclipse_raw

#devices3 = [2106, 2108, 2129, 2060, 2077, 2093, 2069]
alldevice = [2002 2003 2004 2005 2006 2008 2009 2010 2012 2013 2014 2017 2038 2037 2042 2043 2046 2048 2049 2050 2052 2053 2054 2055 2056 2058 2060 2062 2063 2064 2065 2066 2067 2068 2069 2070 2072 2073 2074 2076 2077 2079 2080 2081 2082 2083 2084 2090 2091 2092 2093 2094 2095 2096 2097 2098 2099 2100 2101 2102 2103 2104 2105 2106 2107 2108 2110 2111 2112 2113 2115 2116 2118 2119 2120 2121 2122 2123 2124 2125 2127 2128 2129 2130 2131 2132 2134 2135 2136 2138 2139 2142 2143 2144 2150 2162 2172 2174 2175 2176 2177 2179 2182 2183 2184 2185 2191 2207 2212 2047 2137 2208 2209]
raw = get_eclipse_data(alldevice,"2022-02-01","2021-03-01")
raw.to_csv('../calibrated_no2_full_data-Feb2022.csv')

# Read in data if already pulled

In [9]:
df = pd.read_csv('../calibrated_no2_full_data-Feb2022.csv',index_col = 0)
#df = pd.read_csv('calibrated_no2_full_data-Aug2021.csv',index_col = 0)

In [10]:
df['DeviceId'] = df['MSRDeviceNbr']
df['ReadingDateTimeUTC'] = pd.to_datetime(df.ReadingDateTimeUTC)
# drop zero values
df = df[df['CalibratedPM25']>0].reset_index(drop=True)
df = df[df['CalibratedNO2']>0].reset_index(drop=True)

In [11]:
df = df.sort_values('ReadingDateTimeUTC').reset_index(drop=True)

# Clean eclipse data

Resample for 1 hour measurements, drop when less that 75% of hourly data is available

In [12]:
#
st_dt ='2022-02-01'
end_dt ='2022-02-28 23:00:00'


# Clean data given the number of readings -- if less than 12 readings in an hour drop
df2 = df[['CalibratedPM25','ReadingDateTimeUTC','DeviceId']]
df2s = df2.groupby(["DeviceId"]).resample("H",on="ReadingDateTimeUTC").count()
under_count = (df2s.DeviceId >= 9) # use this as drop index

# drop given the restrictions
ts = df.groupby(["DeviceId"]).resample("H",on="ReadingDateTimeUTC").mean()[under_count]
#print(ts)

# reindex ts so that nan values fill up missing time steps
ts = ts.drop(columns='DeviceId').reset_index()
t_index = pd.DatetimeIndex(pd.date_range(start= st_dt, end= end_dt, freq='1h'))
ids = df.DeviceId.unique()

x,y = [],[] # for later use for cmaq
ts_nomissing = pd.DataFrame()
missing_ids = []
# create dataframe that contains every hour of data
for i in range(len(ids)):
  if len(ts[ts['DeviceId'] == ids[i]]) > 0:
    tmp = ts[ts['DeviceId'] == ids[i]].reset_index(drop=True)
    xx = tmp.Longitude[0]; yy = tmp.Latitude[0]
    x.append(xx); y.append(yy)
    # reindex with complete hourly spread
    tmp = tmp.set_index(pd.to_datetime(tmp['ReadingDateTimeUTC']))
    tmp = tmp.resample('1h').mean().reindex(t_index).fillna(np.nan)
    # Backfill static data
    tmp['DeviceId'] = [ids[i]]*len(tmp)
    tmp['Latitude'] = [yy]*len(tmp)
    tmp['Longitude'] = [xx]*len(tmp)
    tmp['latlon'] = [(xx,yy)]*len(tmp)
    ts_nomissing = ts_nomissing.append(tmp)
  else: missing_ids.append(ids[i])

ts_nomissing

,DeviceId,Unnamed: 0.1,Latitude,Longitude,CO,SO2,NO2,O3,TempC,Humidity,...,PMConc05,PMConc1,PMConc25,PMConc10,PMTypSize,OutlierInd,CalibratedPM25,CalibrationVersion,CalibratedNO2,latlon
2022-02-01 00:00:00,2110,433772.5,41.692519,-87.642557,0.422914,92.882247,114.822872,-10.843734,-0.506325,61.624400,...,304.335396,375.295272,389.482029,392.796977,0.761068,0.0,48.810000,1.0,28.130000,"(-87.642557, 41.692519)"
2022-02-01 01:00:00,2110,433760.5,41.692519,-87.642557,0.332283,91.120203,116.458009,-10.797684,-0.763563,62.828827,...,175.294201,211.202777,216.373411,217.556784,0.743783,0.0,27.961667,1.0,24.754167,"(-87.642557, 41.692519)"
2022-02-01 02:00:00,2110,433749.0,41.692519,-87.642557,0.251231,87.363045,116.586483,-10.817878,-0.420026,64.243247,...,160.998187,196.076213,202.093493,203.487209,0.752386,0.0,26.261818,1.0,21.800909,"(-87.642557, 41.692519)"
2022-02-01 03:00:00,2110,433737.5,41.692519,-87.642557,0.245910,90.350193,112.452745,-10.826465,-0.434672,64.408112,...,115.184524,139.949620,144.054443,145.003129,0.748208,0.0,20.271667,1.0,20.986667,"(-87.642557, 41.692519)"
2022-02-01 04:00:00,2110,433726.0,41.692519,-87.642557,0.239664,89.336506,114.798491,-10.848631,-0.402790,65.685203,...,115.837002,139.038997,142.137266,142.842179,0.706880,0.0,19.320909,1.0,19.657273,"(-87.642557, 41.692519)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-28 19:00:00,2005,19841.0,41.852400,-87.695200,0.131526,25.941184,90.667388,0.496457,14.273356,36.533564,...,42.904932,50.097886,51.171208,51.411425,0.611187,0.0,10.376364,1.1,10.821818,"(-87.6952, 41.852399999999996)"
2022-02-28 20:00:00,2005,19830.5,41.852400,-87.695200,0.155911,18.285838,93.188930,1.046049,15.443840,32.392731,...,68.024410,81.618435,84.641723,85.319606,0.682605,0.0,14.607000,1.1,13.826000,"(-87.6952, 41.852399999999996)"
2022-02-28 21:00:00,2005,19820.0,41.852400,-87.695200,0.300000,30.113839,93.212091,0.563131,14.235972,34.277482,...,45.870443,55.375696,57.618646,58.121669,0.688906,0.0,13.629091,1.1,22.442727,"(-87.6952, 41.852399999999996)"
2022-02-28 22:00:00,2005,NaN,41.852400,-87.695200,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(-87.6952, 41.852399999999996)"


In [30]:
drop_ids = []
# identify sensors with < 20 days of data
for i in range(len(ids)):
    tmp = ts_nomissing[ts_nomissing.DeviceId == ids[i]]        
    if np.sum(~np.isnan(tmp['CalibratedNO2'])) < (len(tmp)*.75):
        drop_ids.append(ids[i])


print(len(tmp)*(.75))
print(len(drop_ids))

for i in range(len(drop_ids)):
    ts_nomissing = ts_nomissing[ts_nomissing.DeviceId != drop_ids[i]].reset_index(drop=True)

ts_nomissing

504.0
18


,DeviceId,Unnamed: 0.1,Latitude,Longitude,CO,SO2,NO2,O3,TempC,Humidity,...,PM25_NO3,PM25_OC,PM25_SOIL,PM25_TOT,RH,SFC_TMP,U10,V10,VOC,precip
0,2110,433772.5,41.692519,-87.642557,379.085999,1.888436,22.337954,19.638380,-0.506325,61.624400,...,8.612284,5.850229,4.028581,30.193928,56.167244,-0.960693,-2.073394,2.028235,81.232269,13.017584
1,2110,433760.5,41.692519,-87.642557,306.648407,1.709394,18.597445,21.988743,-0.763563,62.828827,...,8.272737,5.585948,3.447805,27.981869,54.093136,-1.166595,-2.117607,2.434937,70.904221,13.017584
2,2110,433749.0,41.692519,-87.642557,223.317184,1.172251,13.531857,26.858986,-0.420026,64.243247,...,5.597437,4.982153,2.756583,22.031319,52.089245,-0.982269,-1.845525,2.717877,52.742138,13.018102
3,2110,433737.5,41.692519,-87.642557,173.781204,0.866592,9.881525,30.105042,-0.434672,64.408112,...,3.920908,4.601271,2.276035,18.301626,51.223248,-0.664398,-1.776941,3.008723,40.593750,13.020221
4,2110,433726.0,41.692519,-87.642557,151.540924,0.810289,8.086990,30.569338,-0.402790,65.685203,...,3.328107,4.189316,2.019040,16.344429,52.620644,-0.876190,-1.608718,3.212271,35.813900,13.020977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54427,2122,484464.5,41.907237,-87.667450,187.353226,1.161368,8.784961,47.210705,13.207607,38.250987,...,4.062373,1.992491,2.111834,13.299279,32.473610,7.856628,1.065458,3.723716,39.106194,103.146843
54428,2122,484453.0,41.907237,-87.667450,186.167023,1.528380,9.535040,47.698246,14.363417,33.412448,...,4.312344,1.960625,2.387036,14.242504,25.919584,9.538208,1.839942,3.929317,39.306797,103.146843
54429,2122,484441.5,41.907237,-87.667450,236.252121,1.384298,15.053937,43.652649,13.889643,35.235850,...,3.187061,1.953722,2.688255,13.258142,23.745136,10.767670,2.350792,2.961521,54.044193,103.146843
54430,2122,484430.0,41.907237,-87.667450,297.313232,1.447433,21.984911,36.636028,12.306082,40.774536,...,1.989771,2.045493,3.256510,12.861075,23.856852,10.914764,2.086752,2.266074,70.132225,103.146843


In [ ]:
ts_nomissing.to_csv('../dataAnalysis/cleaned_and_dropped_wint.csv')

# 

# Now append cmaq to eclipse data

Find corresponding indices that match eclipse lon/lat to cmaq lon/lat; pull data from cmaq; add these as columns in eclipse data

In [15]:
def find_index(stn_lon, stn_lat, wrf_lon, wrf_lat):
# stn -- list (points)
# wrf -- list (grid)
  xx=[];yy=[]
  for i in range(len(stn_lat)):
    abslat = np.abs(wrf_lat-stn_lat[i])
    abslon= np.abs(wrf_lon-stn_lon[i])
    c = np.maximum(abslon,abslat)
    latlon_idx = np.argmin(c)
    x, y = np.where(c == np.min(c))
    #add indices of nearest wrf point station
    xx.append(x);yy.append(y)
  #return indices list
  return xx, yy

from netCDF4 import Dataset

# Append CMAQ data
d = '../netcdf/'
lat,lon = np.array(Dataset(d+'/latlon_ChicagoLADCO_d03.nc')['lat']),np.array(Dataset(d+'/latlon_ChicagoLADCO_d03.nc')['lon'])

ds = Dataset('/mnt/batch/tasks/shared/LS_root/mounts/clusters/cmaq/code/Users/t-stacym/netcdf/all_crop_wint.nc')

#var_list = ['BENZENE','CO','FORM','NO', 'NO2', 'NOX', 'O3', 'PBLH',
# 'PM25_CA', 'PM25_CL', 'PM25_EC', 'PM25_HP', 'PM25_K', 'PM25_MG', 'PM25_NA', 'PM25_NH4', 'PM25_NO3', 
# 'PM25_OC', 'PM25_OM', 'PM25_SO4', 'PM25_SOIL', 'PM25_TOT',
# 'RH', 'SFC_TMP','SO2', 'U10', 'V10', 'VOC', 'precip']

# variables to add to dataframe
var_list = ['CO','FORM','NO', 'NO2', 'NOX', 'O3', 'PBLH',
 'PM25_EC', 'PM25_HP',  'PM25_NO3', 'PM25_OC', 'PM25_SOIL', 'PM25_TOT',
 'RH', 'SFC_TMP','SO2', 'U10', 'V10', 'VOC', 'precip']


ids = ts_nomissing.DeviceId.unique()

# find corresponding index in cmaq data for each station
x, y =[],[]
for i in ids:
  tmp = ts_nomissing[ts_nomissing.DeviceId == i].reset_index(drop=True)
  x.append(tmp['Longitude'][0])
  y.append(tmp.Latitude[0])


xx, yy = find_index(x,y,lon,lat) # get index of ID in cmaq file
txx,tyy = [],[]
#print(xx)

In [19]:
# get cmaq data to given index locations

# for each var
for var in var_list:
  cmaq = []
  # for each ID from eclipse
  for i in range(len(ids)):
    ind = xx[i][0],yy[i][0]
    # for each dat
    for d in range(28): # change this if doing august or february
      # for all hours
      for h in range(24):
        cmaq.append(ds[var][d*24+h][0][ind])
          if var == var_list[0]:
            txx.append(ind)
  ts_nomissing[var] = cmaq
  print('Done with var %s'%(var))



Done with var CO
Done with var FORM
Done with var NO
Done with var NO2
Done with var NOX
Done with var O3
Done with var PBLH
Done with var PM25_EC
Done with var PM25_HP
Done with var PM25_NO3
Done with var PM25_OC
Done with var PM25_SOIL
Done with var PM25_TOT
Done with var RH
Done with var SFC_TMP
Done with var SO2
Done with var U10
Done with var V10
Done with var VOC
Done with var precip


In [20]:
#reindexing arrays, keep it from look weird
f = ds[var][d*24:d*24+24].reshape(288,315,24,1)[ind]
f2 = [ds[var][d*24+h][0][ind] for h in range(24)]

[[59.404232]
 [59.096165]
 [59.268177]
 [59.940365]
 [61.810226]
 [64.50386 ]
 [67.14056 ]
 [69.7813  ]
 [73.13299 ]
 [76.5077  ]
 [78.53136 ]
 [78.66185 ]
 [77.85525 ]
 [77.16622 ]
 [76.634575]
 [76.20534 ]
 [76.7999  ]
 [79.3293  ]
 [84.17417 ]
 [88.68583 ]
 [90.23093 ]
 [89.72246 ]
 [88.639435]
 [87.891975]]
[100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652, 100.44652]


In [27]:
#what does this look like in the end
ts_nomissing.Latitude

2022-02-01 00:00:00    41.692519
2022-02-01 01:00:00    41.692519
2022-02-01 02:00:00    41.692519
2022-02-01 03:00:00    41.692519
2022-02-01 04:00:00    41.692519
                         ...    
2022-02-28 19:00:00    41.852400
2022-02-28 20:00:00    41.852400
2022-02-28 21:00:00    41.852400
2022-02-28 22:00:00    41.852400
2022-02-28 23:00:00    41.852400
Name: Latitude, Length: 66528, dtype: float64

In [28]:
#ts_nomissing['xx'] = txx
# put out the file so that we can tmp use it
ts_nomissing.to_csv('cleaned_data_wint.csv')

# Now check nearby sensors to identify anticorrelated sensors

In [ ]:
df = pd.read_csv('../cleaned_data_wint.csv',index_col = 0)
df['DeviceId'] = df['MSRDeviceNbr']
df2 = df.groupby('DeviceId').mean().reset_index()

ids = df.DeviceId.unique()

lo,la = [],[]

for id in ids:
    lo.append(df[df.DeviceId == id].Longitude.unique()[0])
    la.append(df[df.DeviceId == id].Latitude.unique()[0])


geometry = [Point(lo[i],la[i]) for i in range(len(lo))]

id = [ids[i] for i in range(len(ids))]
f = pd.DataFrame([id,lo,la]).T
f.columns = ['ids','lon','lat']

eclipse = gpd.GeoDataFrame(f,geometry = geometry,crs ={'init': 'epsg:4326'})
eclipse = eclipse.to_crs("EPSG:3857")

match = []
match_long = []
for i in range(len(ids)):
    match = []
    for j in range(len(eclipse)):
        point_1 = eclipse.geometry[i]
        point_2 = eclipse.geometry[j]
        d = point_1.distance(point_2)
        if (d < 5000) & (d != 0):
            match.append(int(eclipse.ids[j]))
            #print(match)
    match_long.append(match)

f = pd.DataFrame(match_long)
f['id'] = id
f.to_csv('nearest_station_names.csv')

In [ ]:
data = pd.read_csv('../cleaned_data_wint.csv')
ids = data.DeviceId.unique()
loc = []

for id in ids:
    loc.append([data[data.DeviceId == id].Longitude.unique()[0],data[data.DeviceId == id].Latitude.unique()[0]])

import warnings 
warnings.filterwarnings('ignore')

df['dt'] = pd.to_datetime(df['Unnamed: 0'])

times = df.dt.unique()
times = pd.date_range(times[0],times[-1])
df['day_dt'] = [df['Unnamed: 0'][i][0:10] for i in range(len(df))]
df['day_dt'] = pd.to_datetime(df['day_dt'])

rs = []
for id in ids:
    tmp = df[df.DeviceId == id].reset_index()
    closest = np.array(f[f.id == id])[0]
    n = (np.sum(~np.isnan(closest))) # number of close sensors
    r = []
    if n > 1:
        for j in range(n-1):
            ri = []
            tmp_closest = df[df.DeviceId == closest[j]].reset_index()
            for i in range(len(times)):
                t = np.array(tmp[tmp.day_dt == times[i]]['CalibratedNO2'])
                t2 = np.array(tmp_closest[tmp_closest.day_dt == times[i]]['CalibratedNO2'])
                if (len(t2) > 0) & (len(t) > 0):
                    m1= np.array([~np.isnan(t),~np.isnan(t2)])
                    m1 = m1.all(0)
                    #print(m1)
                    t,t2 = t[m1],t2[m1]
                    ri.append(np.corrcoef(t,t2)[0][1])
            r.append(ri)    
        #print(np.array(r).shape)
        #print(r)
        rs.append(list(np.nanmean(np.array(r),axis=0)))
    else:
        rs.append([np.nan]*len(times))

pd.DataFrame(rs)

In [ ]:
check = pd.DataFrame(
    [ids,
    list(pd.DataFrame(rs).T.mean()),
    list(pd.DataFrame(rs).T.min()),
    list(pd.DataFrame(rs).T.max())]
    ).T

check

In [ ]:
rs = pd.DataFrame(rs)
rs['Correlated Days'] = np.array(rs > 0).sum(axis=1)-1
a = np.sort(rs[rs['Correlated Days'] < 31*0.75].id)
a
b = np.array([2118, 2080, 2134, 2060, 2082, 2062, 2079, 2048, 2172, 2182, 2179, 2191, 2162]) # form hourly concentraitons per month
np.sort(b)

s = list(a)+list(b)
s = pd.DataFrame([s,s]).T[0].unique()
print(len(s))
print(s)

In [ ]:
locs = pd.DataFrame(loc)
locs['id'] = ids


fig,ax = plt.subplots(figsize=(9,12))
ax.scatter(locs[0],locs[1])

for i in range(len(s)):
    tmp = locs[locs.id == s[i]]
    ax.scatter(tmp[0],tmp[1])

plt.show()

In [ ]:
# drop sensors bad
for i in range(len(s)):
    indexNames = data[data['DeviceId'] == s[i]].index
    data.drop(indexNames , inplace=True)

data.reset_index(drop=True)
data.to_csv('data_cleaned_and_dropped_wint.csv')

In [ ]:
fin = pd.read_csv('data_cleaned_and_dropped.csv',index_col = 0)
fin = fin.groupby(['Latitude','Longitude']).mean().reset_index()
chi_rect_shp = gpd.read_filegrid = gpd.read_file('../shapefiles/rectangle_chicago_1km_grid.shp')
chi_shp = gpd.read_file('../shapefiles/geo_export_77af1a6a-f8ec-47f4-977c-40956cd94f97.shp')

# for plotting
streets = gpd.read_file('../shapefiles/geo_export_d268c11a-e3fe-42de-a88f-ad64688102f5.shp')
class1 = streets[streets['class'] == '1'].reset_index(drop=True)

chi_shp = chi_shp.to_crs("EPSG:4326")
outer = gpd.GeoDataFrame({'index':[1],'geometry':chi_rect_shp.unary_union},crs='EPSG:4326')
outer_chi = gpd.GeoDataFrame({'index':[1],'geometry':chi_shp.unary_union[0]},crs='EPSG:4326')

class1 = class1.to_crs(chi_shp.crs)


In [ ]:
# make figure 1

import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize = (10,10))

outer.plot(facecolor='None',edgecolor='k',alpha=0.5,ax=ax,linewidth=1)
outer_chi.plot(facecolor='None',edgecolor='k',alpha=0.5,ax=ax,linewidth=1)

chi_shp.plot(facecolor='None',edgecolor='k',alpha=0.5,ax=ax,linewidth=0.5)
class1.plot(edgecolor='midnightblue',alpha=0.5,ax=ax,linewidth=1.5)

ctx.add_basemap(ax,crs=chi_shp.crs,source=ctx.providers.CartoDB.PositronNoLabels) #ctx.providers.Esri.NatGeoWorldMap)

ax.scatter(df2.Longitude, df2.Latitude,color='slategray',alpha=0.7,s=15,label='Excluded Station',marker='^')
ax.scatter(fin.Longitude, fin.Latitude,color='royalblue',s=20,label=r'NO$_2$ Sensor')


# add label
ax.text(0.10,0.96,s='(a)',transform=ax.transAxes,fontsize = 12,c='k',alpha=0.8)
ax.text(0.22,0.87,s='(b)',transform=ax.transAxes,fontsize = 12,alpha=0.8)

ax.text(0.14,0.795,s="O'Hare\n   ✈",transform=ax.transAxes,fontsize = 10,alpha=0.8)
ax.text(0.4,0.45,s='Midway\n   ✈',transform=ax.transAxes,fontsize = 10,alpha=0.8)
ax.text(0.45,0.14,s='EPA ComED',transform=ax.transAxes,fontsize = 10,alpha=0.8)

ax.text(0.74,0.35,s="I-90",transform=ax.transAxes,fontsize = 9,alpha=0.8, style='italic')
ax.text(0.53,0.6,s='I-290',transform=ax.transAxes,fontsize = 9,alpha=0.8, style='italic')
ax.text(0.53,0.48,s='I-55',transform=ax.transAxes,fontsize = 9,alpha=0.8, style='italic')
ax.text(0.45,0.75,s='I-94',transform=ax.transAxes,fontsize = 9,alpha=0.8, style='italic')
ax.text(0.65,0.35,s='I-94',transform=ax.transAxes,fontsize = 9,alpha=0.8, style='italic')
ax.text(0.58,0.2,s='I-57',transform=ax.transAxes,fontsize = 9,alpha=0.8, style='italic')
ax.text(0.75,0.2,s='I-94',transform=ax.transAxes,fontsize = 9,alpha=0.8, style='italic')
ax.text(0.72,0.58,s='LSD',transform=ax.transAxes,fontsize = 9,alpha=0.8, style='italic')

ax.text(0.8,0.7,s='  Lake\nMichigan',transform=ax.transAxes,fontsize = 12,alpha=0.5, style='italic')


plt.legend()

plt.show()

In [166]:
#ts_nomissing = ts_nomissing.reset_index(inplace=True)
err = [2048,2172,2182,2179,2191,2964,2062,2082,2079,2176,2069,2080]
#ts_nomissing.iloc(ts_nomissing.index([ts_nomissing['MSRDeviceNbr'] == err[0]]))['CalibratedNO2'] = np.nan
ts_nomissing
#ts_nomissing.index([ts_nomissing['MSRDeviceNbr'] == err[0]])

,DeviceId,Unnamed: 0.1,Latitude,Longitude,CO,SO2,NO2,O3,TempC,Humidity,...,PM25_OC,PM25_SOIL,PM25_TOT,RH,SFC_TMP,U10,V10,VOC,precip,xx
2021-08-01 00:00:00,2096,451566.5,41.7136,-87.5602,223.072906,0.598424,7.072208,69.273613,26.704801,49.475733,...,4.255794,2.850836,14.695996,40.932316,26.526184,-2.705408,-0.099267,45.112770,0.070998,"(106, 148)"
2021-08-01 01:00:00,2096,451554.5,41.7136,-87.5602,249.280609,1.770328,15.038789,55.888779,25.407931,54.117076,...,4.377907,4.122512,18.598806,44.280441,25.108826,-1.739362,0.138814,64.181183,0.070998,"(106, 148)"
2021-08-01 02:00:00,2096,451543.0,41.7136,-87.5602,264.847870,3.923457,20.209375,34.969601,24.262903,58.412170,...,3.386872,5.435637,20.073183,47.657852,23.516479,-0.961102,1.435569,79.380402,0.070998,"(106, 148)"
2021-08-01 03:00:00,2096,451531.5,41.7136,-87.5602,197.811447,0.715859,13.738832,27.680510,23.346691,62.313334,...,1.727185,4.240244,14.624983,59.108917,22.568420,1.822631,1.104520,64.052742,0.070998,"(106, 148)"
2021-08-01 04:00:00,2096,451519.5,41.7136,-87.5602,167.599396,0.568978,10.944631,27.643175,22.998441,65.065257,...,1.522062,3.927935,11.371109,65.886513,21.867828,1.990919,1.212685,51.081818,0.070998,"(106, 148)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31 19:00:00,2162,NaN,41.8454,-87.6854,148.654388,0.968956,3.183031,47.529999,NaN,NaN,...,1.105391,0.654756,5.329491,48.975044,26.728271,-3.747720,-1.621235,18.894489,89.446609,"(116, 140)"
2021-08-31 20:00:00,2162,NaN,41.8454,-87.6854,152.744156,0.380541,3.196157,42.690430,NaN,NaN,...,0.914257,0.640529,3.972908,47.038013,26.793213,-3.807886,-1.849999,19.979692,89.446609,"(116, 140)"
2021-08-31 21:00:00,2162,NaN,41.8454,-87.6854,159.451935,0.268171,3.513601,40.298920,NaN,NaN,...,0.791802,0.667005,3.482544,47.418720,26.687378,-3.795038,-2.354437,21.347155,89.446609,"(116, 140)"
2021-08-31 22:00:00,2162,812023.0,41.8454,-87.6854,159.451935,0.268171,3.513601,40.298920,28.807005,45.623224,...,0.791802,0.667005,3.482544,47.418720,26.687378,-3.795038,-2.354437,21.347155,89.446609,"(116, 140)"


In [102]:
# drop errenous sensors
err = [2048,2172,2182,2179,2191,2964,2062,2082,2079,2176,2069,2080]
for e in err:
    ts_nomissing.at([ts_nomissing['MSRDeviceNbr'] == e]['CalibratedNO2']) = np.nan


SyntaxError: cannot assign to function call (<ipython-input-102-015717c94922>, line 4)

In [ ]:
ts_nomissing.to_csv('cleaned_data_wint.csv')

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, shape, Polygon

# Plot the monthly averages

In [ ]:
monthly_avg = ts_nomissing.groupby('DeviceId').resample('M').mean().reset_index(drop=True)
monthly_avg['geometry'] = [Point(monthly_avg.Longitude[i],monthly_avg.Latitude[i]) for i in range(len(monthly_avg))]
monthly_avg = gpd.GeoDataFrame(monthly_avg)
monthly_avg=monthly_avg.set_crs("EPSG:4326")

In [ ]:
ts_nomissing['CalibratedPM25']

In [ ]:
from shapely.geometry import mapping
import rioxarray as rxr
import xarray as xr
import geopandas as gpd

In [ ]:
f = gpd.read_file('shapefiles/tmp_clipped.shp')
f = f.to_crs(monthly_avg.crs)
hwy = gpd.read_file('shapefiles/hwy.shp')
hwy = hwy.to_crs(monthly_avg.crs)

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
f.plot('pm25',ax=ax,vmin=10,vmax=15)
hwy.plot(color='k',ax=ax,linewidth=0.25)
monthly_avg.plot('CalibratedPM25',vmin=10,vmax=15,ax=ax,s=20,edgecolor='white',linewidth=0.25,zorder=10,
                legend=True,legend_kwds={'fraction':0.03, 'pad':0.04, 'label':r'${\mu}g/m^3$'})
ax.axis('off')
ax.set_title(r'Monthly Average PM$_{2.5}$ over Chicago')
plt.show()

fig,ax = plt.subplots(figsize=(10,10))
f.plot('no2',ax=ax,vmin=10,vmax=20)
hwy.plot(color='k',ax=ax,linewidth=0.25)
monthly_avg.plot('CalibratedNO2',vmin=10,vmax=20,ax=ax,s=20,edgecolor='white',linewidth=0.25,zorder=10,
                legend=True,legend_kwds={'fraction':0.03, 'pad':0.04, 'label':r'${\mu}g/m^3$'})
ax.axis('off')
ax.set_title(r'Monthly Average NO2$_{2.5}$ over Chicago')
plt.show()

In [ ]:
ts_nomissing.to_csv('cleaned_data.csv')

In [ ]:

print(df[df['LocationName'] == 'EPA COM ED Maintenance Bldg D']['DeviceId'].unique()) # 2008
print(df[df['LocationName'] == 'EPA COM ED Maintenance Bldg C']['DeviceId'].unique()) #2017
print(df[df['LocationName'] == 'EPA COM ED Maintenance Bldg E']['DeviceId'].unique()) #2208

print(df[df['LocationName'] == 'EPA Springfield Pump Station C']['DeviceId'].unique()) #2010
print(df[df['LocationName'] == 'EPA Springfield Pump Station B']['DeviceId'].unique()) #2009
print(df[df['LocationName'] == 'EPA Springfield Pump Station E']['DeviceId'].unique()) #2212

print(df[df['LocationName'] == 'EPA Village Garage D']['DeviceId'].unique()) #2144
print(df[df['LocationName'] == 'EPA Village Garage C']['DeviceId'].unique()) #2014
print(df[df['LocationName'] == 'EPA Village Garage B']['DeviceId'].unique()) #2013


In [ ]:
df[df['LocationName'] == 'EPA COM ED Maintenance Bldg D']
df[df.DeviceId == missing_ids[12]]


In [ ]:
for i in range(len(missing_ids)):
    print(df[df.DeviceId == missing_ids[i]]['LocationName'].iloc[0])